In [5]:
from seleniumbase import Driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

In [19]:
def open_browser():
    # Create SeleniumBase driver
    driver = Driver(browser="chrome", incognito=True, undetectable=True)

    # Remove webdriver flag
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
            });
            """
        }
    )

    # Custom user agent
    driver.execute_cdp_cmd(
        "Network.setUserAgentOverride",
        {
            "userAgent":
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        }
    )

    return driver
def human_delay(a=0.4, b=1.3):
    time.sleep(random.uniform(a, b))

def login_to_jumia(driver, email, password):
    """Logs into Jumia Nigeria and waits for manual OTP entry if needed."""
    wait = WebDriverWait(driver, 30)
    actions = ActionChains(driver)

    print("Opening Jumia homepage...")
    driver.get("https://www.jumia.com.ng")
    human_delay()

    # Close pop-up if present
    print("Closing popups...")
    actions.send_keys(Keys.ESCAPE).perform()
    human_delay()

    # Click 'Account'
    print("Clicking on Account menu...")
    account_label = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="jm"]/header/section/div[2]/div[1]/label')))
    account_label.click()
    human_delay()

    # Click 'Sign In'
    print("Clicking on Sign In...")
    sign_in_link = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="dpdw-login-box"]/div/div/a')))
    sign_in_link.click()
    human_delay()

    # ✅ Wait for page transition (30–40 seconds)
    print("⏳ Waiting 30 seconds for the login page to load fully...")
    time.sleep(30)
    
    # Enter email
    print("Entering email...")
    email_input = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="input_identifierValue"]')))
    email_input.send_keys(email)
    human_delay()

    # Click 'Continue'
    print("Clicking Continue...")
    continue_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="identification-form"]/div[1]/div[3]/div[2]/button/span[3]')))
    continue_btn.click()

    # Input Password (30–40 seconds)
    print("⏳ Waiting 20 seconds to input password")
    time.sleep(10)

    # Enter password
    print("Entering password...")
    password_input = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="passwordForm"]/div/div[3]/div/label/input')))
    password_input.send_keys(password)

    # Click 'Login'
    print("Clicking Login...")
    login_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="loginButton"]/span[3]')))
    login_btn.click()

    # Wait for possible OTP
    time.sleep(30)
    if "otp" in driver.current_url.lower():
        print("⚠️ OTP page detected. Please manually enter your OTP in the browser.")
        input("⏸ Press ENTER here *after* OTP is entered and you’re redirected to homepage...")

    # Wait until redirect to homepage
    wait.until(EC.url_contains("jumia.com.ng"))
    print("✅ Login successful and homepage loaded.")


def search_product(driver, product_name):
    """Search for a product and open the first result."""
    wait = WebDriverWait(driver, 10)
    print(f"Searching for '{product_name}'...")

    search_box = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="fi-q"]')))
    search_box.clear()
    search_box.send_keys(product_name)

    search_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search"]/button')))
    search_btn.click()

    print("Waiting for search results...")
    time.sleep(5)

    # Click on the first product (if present)
    first_product = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="jm"]/main/div[2]/div[3]/section/div[2]/article[1]/a')))
    driver.execute_script("arguments[0].scrollIntoView();", first_product)
    first_product.click()

    print("✅ Opened first product page.")


def add_to_cart(driver):
    """Adds the opened product to the cart."""
    wait = WebDriverWait(driver, 15)

    print("Adding product to cart...")
    add_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="add-to-cart"]')))
    driver.execute_script("arguments[0].scrollIntoView();", add_btn)
    add_btn.click()
    human_delay()

    print("✅ Product added to cart.")


def go_to_cart_and_checkout(driver):
    """Navigates to cart and clicks checkout."""
    wait = WebDriverWait(driver, 15)

    print("Opening cart...")
    cart_icon = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="jm"]/header/section/div[2]/a')))
    cart_icon.click()
    human_delay()

    print("Proceeding to checkout...")
    checkout_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="jm"]/main/div/div[2]/div/article/div[2]/a')))
    checkout_btn.click()

    print("✅ Checkout page reached.")
    input("⏸ Press ENTER to end automation and close browser...")


def main():
    YOUR_EMAIL = "madwisdom25@gmail.com"       
    YOUR_PASSWORD = "Blessing@25"             
    PRODUCT_NAME = "Macbook"                  

    driver = open_browser()

    try:
        login_to_jumia(driver, YOUR_EMAIL, YOUR_PASSWORD)
        search_product(driver, PRODUCT_NAME)
        add_to_cart(driver)
        go_to_cart_and_checkout(driver)

    except Exception as e:
        print(f"❌ Error: {e}")

    finally:
        driver.quit()
        print("🚪 Browser closed.")


if __name__ == "__main__":
    main()

Opening Jumia homepage...
Closing popups...
Clicking on Account menu...
Clicking on Sign In...
⏳ Waiting 30 seconds for the login page to load fully...
Entering email...
Clicking Continue...
⏳ Waiting 20 seconds to input password
Entering password...
Clicking Login...
✅ Login successful and homepage loaded.
Searching for 'Macbook'...
Waiting for search results...
✅ Opened first product page.
Adding product to cart...
✅ Product added to cart.
Opening cart...
Proceeding to checkout...
✅ Checkout page reached.


⏸ Press ENTER to end automation and close browser... 


🚪 Browser closed.
